In [1]:
import ee
import numpy as np
import os
import palettable 
import numpy as np
from Satellite import *
from Simulator import *

service_account = "jims-948@jimssatellites.iam.gserviceaccount.com"
key_path = 'jimssatellites-25b0240d5cf2.json'
credentials = ee.ServiceAccountCredentials(service_account, key_path)
ee.Initialize(credentials=credentials)
import geemap
import folium



sat1 = Satellite("Satellites/sat1.txt", "SAT1")
sat2 = Satellite("Satellites/sat2.txt", "SAT2")
sat3 = Satellite("Satellites/sat3.txt", "SAT3")
sat4 = Satellite("Satellites/sat4.txt", "SAT4")
sat5 = Satellite("Satellites/sat5.txt", "SAT5")
sat6 = Satellite("Satellites/sat6.txt", "SAT6")  
sat7 = Satellite("Satellites/sat7.txt", "SAT7")
sat8 = Satellite("Satellites/sat8.txt", "SAT8")
sat9 = Satellite("Satellites/sat9.txt", "SAT9")
sat10 = Satellite("Satellites/sat10.txt", "SAT10")
sat11 = Satellite("Satellites/sat11.txt", "SAT11")
sat12 = Satellite("Satellites/sat12.txt", "SAT12")
sim = Simulator([sat1,sat2,sat3,sat4,sat5,sat6,sat7,sat8,sat9,sat10,sat11,sat12], [])
sim.simulate(0,6*60*60, 1, motionEquation)
mappingArray= [] 


Simulating :  10%|█         | 10.324074074074293/100 [1:59:24<10:15:45, 411.99s/it]    

In [34]:

count = 0

for sat in sim.satellites:
    for image in  sat.payload.images:
        coord1 = list(image[0][0:2])
        coord2 = list(image[1][0:2])
        coord3 = list(image[2][0:2])
        coord4 = list(image[3][0:2])
        coord1.reverse()
        coord2.reverse()
        coord3.reverse()
        coord4.reverse()
        bounding = [coord1[0], coord1[1],  coord4[0], coord4[1]]

        roi = ee.Geometry.Rectangle(bounding)
        newFeature = ee.Feature(roi, {'name': str(count)})

        count = count+1
        mappingArray.append(newFeature)

# Create an interactive map
Map = geemap.Map(center=[0, 0], zoom=2)
Map.add_basemap("NASAGIBS.BlueMarble")
#Get rid of text
Map.setOptions(ui_inspector_enabled=False, ui_layers=False)


#Create region of interest
roi = ee.Geometry.Rectangle([ 146,-35.36,  158.6, -30.83])
#I have no idea why, but for some reason this just paints everything so I'm gonna cop for now
mask = ee.Image(0).toInt8().paint(roi, 1)
fc = ee.FeatureCollection(mappingArray)

# Fill and outline the polygons in two colors
filled = mask.paint(fc, 2)
both = filled.paint(fc, 1, 5)
# Mask off everything that matches the fill color.
result = both.mask(filled.neq(2))

TERN = ee.ImageCollection('TERN/AET/CMRSET_LANDSAT_V2_2')






out_dir = os.path.expanduser('~/Downloads/GEEBAM')
GB = os.path.join(out_dir, 'GEEBAM_v3p1_dNBR_Classes.tif')
# Add layer to map
Map.add_raster(GB, band =1, palette = 'inferno',  layer_name = "GEEBAM")


#Rain fall data
dataset = ee.ImageCollection('TERN/AET/CMRSET_LANDSAT_V2_2')


visualization = {
    'bands': ['ETa'],
    'min': 0,
    'max': 7,
    'palette': ['d7191c', 'fdae61', 'ffffbf', 'abd9e9', '2c7bb6']
}
# Add the layer to the map
#Map.addLayer(dataset, visualization, 'Average daily evapotranspiration (mm/day)')


Map.addLayer(result, {
    'palette': '000000,FF0000',
    'max': 1,
    'opacity': 1.0
}, 'Opaque White Rectangle')




collection = ee.ImageCollection([])

Map


The provided color (000000,FF0000) is invalid. Using the default black color.
'#000000,FF0000' is not in web format. Need 3 or 6 hex digit.


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…